# Imports

In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import shutil
import pandas as pd
from PIL import Image

# URL & Base links

In [2]:
base_url = "https://myanimelist.net/anime.php"
base_genre_url = "https://myanimelist.net/"

main_page_html = requests.get(base_url).content

soup = BeautifulSoup(main_page_html, "lxml")

folder = r"C:\Users\yassi\posters"
folder.replace("\\","/")

'C:/Users/yassi/posters'

# Saving the HTML

In [3]:
# Exporting the HTML to a file

def save_html(html,name):
    with open(name +".html", 'wb') as file:
        file.write(soup.prettify('utf-8'))
    
with open('main_page.html', 'wb') as file:
    file.write(soup.prettify('utf-8'))

# Getting the Links

In [4]:
# sorting_sections: We will get the sorting sections of the MAL (Genres, Explicit, Themes,....)
sorting_sections = soup.find_all('div', attrs = {'class': "normal_header pt24 mb0"})
sorting_sections

# Navigate through the child 'div' to get the links for the Genres 
genres = sorting_sections[0].find_next("div").find_all('a')
genres

# genres_links: partial links for the whole genres
genres_links = [link.get('href') for link in genres]

# raw_genres: whole list of genres
raw_genres = [genre.split('/')[-1] for genre in genres_links]

# full_genres_links: full link of the animes in each genre
full_genres_links = [urljoin(base_genre_url,link) for link in genres_links]
full_genres_links

['https://myanimelist.net/anime/genre/1/Action',
 'https://myanimelist.net/anime/genre/2/Adventure',
 'https://myanimelist.net/anime/genre/5/Avant_Garde',
 'https://myanimelist.net/anime/genre/46/Award_Winning',
 'https://myanimelist.net/anime/genre/28/Boys_Love',
 'https://myanimelist.net/anime/genre/4/Comedy',
 'https://myanimelist.net/anime/genre/8/Drama',
 'https://myanimelist.net/anime/genre/10/Fantasy',
 'https://myanimelist.net/anime/genre/26/Girls_Love',
 'https://myanimelist.net/anime/genre/47/Gourmet',
 'https://myanimelist.net/anime/genre/14/Horror',
 'https://myanimelist.net/anime/genre/7/Mystery',
 'https://myanimelist.net/anime/genre/22/Romance',
 'https://myanimelist.net/anime/genre/24/Sci-Fi',
 'https://myanimelist.net/anime/genre/36/Slice_of_Life',
 'https://myanimelist.net/anime/genre/30/Sports',
 'https://myanimelist.net/anime/genre/37/Supernatural',
 'https://myanimelist.net/anime/genre/41/Suspense']

In [13]:
# add multiple pages of the Genres
        
def all_pages_links(links):
    paged_links = []
    for link in links:
        i=1
        nxt_page= True
        while(nxt_page):
    
            new_link = f"{link}?page={i}"
            req = requests.get(new_link)
            if (int(req.status_code) == 200):
                paged_links.append(new_link)
                i+=1
            else:
                nxt_page = False
                break
    return paged_links
    
paged_full_genres_links = all_pages_links(full_genres_links)

# Saving the htmls of each page as 'html' file

""" 
for page in paged_full_genres_links:
    
    html_link = requests.get(page)
    html_name = page.split('/')[-1]
    save_html(html_link,html_name)
    """
paged_full_genres_links

['https://myanimelist.net/anime/genre/1/Action?page=1',
 'https://myanimelist.net/anime/genre/1/Action?page=2',
 'https://myanimelist.net/anime/genre/1/Action?page=3',
 'https://myanimelist.net/anime/genre/1/Action?page=4',
 'https://myanimelist.net/anime/genre/1/Action?page=5',
 'https://myanimelist.net/anime/genre/1/Action?page=6',
 'https://myanimelist.net/anime/genre/1/Action?page=7',
 'https://myanimelist.net/anime/genre/1/Action?page=8',
 'https://myanimelist.net/anime/genre/1/Action?page=9',
 'https://myanimelist.net/anime/genre/1/Action?page=10',
 'https://myanimelist.net/anime/genre/1/Action?page=11',
 'https://myanimelist.net/anime/genre/1/Action?page=12',
 'https://myanimelist.net/anime/genre/1/Action?page=13',
 'https://myanimelist.net/anime/genre/1/Action?page=14',
 'https://myanimelist.net/anime/genre/1/Action?page=15',
 'https://myanimelist.net/anime/genre/1/Action?page=16',
 'https://myanimelist.net/anime/genre/1/Action?page=17',
 'https://myanimelist.net/anime/genre/1/

In [14]:
len(paged_full_genres_links)

359

# Getting the HTMLs 

In [15]:
def get_htmls(links):
    # animes_htmls_temp: list of html content of each genre link 
    animes_htmls_reqs = [requests.get(link).content for link in links]
    
    # animes_htmls: list of html  of each genre link as bs4 object
    animes_htmls = [BeautifulSoup(html, "lxml") for html in animes_htmls_reqs ]
    
    return animes_htmls

# All animes
animes_htmls = get_htmls(paged_full_genres_links)

# Scraping and Extracting the Data -  Functions

In [18]:
# (1)
def get_genre_discription(animes_htmls):
    
    # Getting the description of each genre webpage
    genres_discriptions = []
    
    for anime in animes_htmls:
        description = anime.find_all('p',attrs = {'class': "pt8 pr8 pb8 pl8 bg-color-base5"})
        genres_discriptions.append(description[0].text)
        
    # genres_discriptions: dictionary of Genre and its description
    genres_discriptions = dict(zip(raw_genres,genres_discriptions))
    
    return genres_discriptions
#(2)
def get_anime_details(anime_html):
    """ 
    Takes an anime: html bs4 file
    returns anime_name:string
            anime_id: string
            anime_link: string
            anime_html: bs4 file
    """
    # anime  header
    anime = anime_html.find_all('h2', attrs = {'class':"h2_anime_title"})[0]
   
    
    # get the link for anime
    
    try:
        link = anime.find_next("div").find('a').get('href')
        anime_link = link.replace("/video","")
        # anime name & ID
        anime_name= link.split('/')[-2]
        anime_id = link.split('/')[-3]
        
    except:
            link = None
            anime_link = None
            anime_name = None
            anime_id = None
    #print(anime.find_next("div").find('a').get('href'))
        
    

    
    
    return anime_name,anime_id,anime_link, anime

#(3)
def get_anime_temporal_details(anime):
    
    """
    Takes an anime: html bs4 file
    returns brd (Method of Broadcast):String
            year(Year of Broadcast): String
            Airing status:  String
            eps (No. Episodes): String
            run_time(Duration of episode): String
    """

    genric_data = anime.find_next('div').find_all('span', class_="item") 
    brd_year = genric_data[0].text
    brd , year = brd_year.split(',')
    brd , year = brd.strip() , year.strip()
    
    # anime status 
    status = genric_data[1].text.strip()
    
    # episodes, run_time 
    eps_run = genric_data[2].text.replace('\n',"")
    eps, run_time = eps_run.split(',')
    eps, run_time = eps.strip() , run_time.strip()
    
    return brd, year, status, eps, run_time

#(4)
def get_anime_related_genre(anime):
    
    """
    Takes an anime: html bs4 file
    returns anime_genre (Other related genre for the anime): list[String]
            anime_genre sub-tree: html bs4 file 
    """
    
    # Anime other related genres
    anime_genre_tree = anime.find_next('div').find_next('div').find_next('div').find_next('div').find_next('div')
    anime_genre = anime_genre_tree.text.strip().split('\n\n')
    
    return anime_genre,anime_genre_tree

#(5)
def get_anime_poster(anime_genre,anime_id,anime_name):
    
    """
    Takes an anime bs4 sub-tree (From the anime_genre tree): html bs4 file
    Saves the anime poster: jpg
    returns poster(Anime poster link): String
            anime_poster sub-tree: html bs4 file 
    """
    # Anime poster link
    anime_poster = anime_genre.find_next('div').find_next('div')
    raw = anime_poster.find_all('a')
    temp_links = str(raw).split('https')
    
    # get the poster link of a list of multiple links
    poster = temp_links[3]
    
    # cut the string from the start till the end of the word ".jpg"
    poster = "https" + poster[:poster.index(".jpg")+4]
    
    
    
    # Saving the poster as 'jpg'
    poster_pic = requests.get(poster, stream=True)
    path = f"{folder}\{anime_id}_{anime_name}.jpg"
    path = path.replace("\\","/")
    with open(path ,'wb') as f:
        shutil.copyfileobj(poster_pic.raw, f)
    
    
    return poster,anime_poster

#(6)
def get_anime_story(anime_poster):
    """
    Takes an anime bs4 sub-tree (From the anime_poster tree): html bs4 file
    returns anime_story: String
            
    """
    # Anime story
    anime_story = anime_poster.find_next('div').find('p').text
    anime_story= anime_story.replace("\r\n\r\n","")
    anime_story= anime_story.replace("[Written by MAL Rewrite]","")
    anime_story= anime_story.replace("\r\n","")
    
    
    return anime_story

#(7)
def get_anime_studio_theme(anime_poster):
    """
    Takes an anime bs4 sub-tree (From the anime_poster tree): html bs4 file
    returns anime_studio: String
            anime_source(Manga,novel,..): String
            anime_theme: list[String]
    """
    # Anime studio, Source, Theme
    std_src_theme = anime_poster.find_next('div').find_next('div').find_all('span', class_="item")
    studio = std_src_theme[0].text
    anime_source = std_src_theme[1].text
    # Theme may contain multiple entries
    anime_theme = [ theme.text for theme in std_src_theme[2:]]
    
    return studio, anime_source, anime_theme

#(8)

def get_anime_score(anime_poster):
    
    """
    Takes an anime bs4 sub-tree (From the anime_poster tree): html bs4 file
    returns anime_score: String

    """
    
    
    score_tree = anime_poster.find_next('div').find_next('div').find_next('div').find_next('div').find_next('div').find_next('div')

    
    
    # Old : score = score_tree.find('div', class_ = "scormem-container").find_next('div').text
    # Old: score = score.replace("\n","").strip()
    #Modified:
    score = score_tree.find_next('div', class_ = "scormem-container").text
    score = score.replace("\n","").strip().split(' ')[0]
    return score

#()



# Scraping and Extracting the Data - Run

In [1]:
#(1) Getting the description of each genre webpage
#genres_discriptions = get_genre_discription(animes_htmls)

In [20]:
# V2
# Extracted data list
raw_data = []
# Page number index
p=0
# Anime number index
a=0

for page in animes_htmls:
    
    animes = page.find_all('div',class_= "title-text")
    
    for anime_html in animes:
        
        #(2) Pass an anime html bs4 object and receive the main details of the anime 
        anime_name,anime_id,anime_link, anime_html_sub = get_anime_details(anime_html)
        anime_main_details = [anime_id, anime_name,anime_link]
        print(f" page({p}), Anime({a}): {anime_name}")

        ids = [row[0] for row in raw_data]

        if (anime_id in ids):
            a+=1
            continue

        #(3) Pass an anime html bs4 object and receive the Temporal details of the anime
        brd, year, status, eps, run_time = get_anime_temporal_details(anime_html_sub)
        anime_temporal_details = [brd, year, status, eps, run_time]

        #(4)
        anime_related_genres, anime_genre = get_anime_related_genre(anime_html_sub)
        all_genres = "_".join(anime_related_genres)
        #(5)
        anime_poster_link,anime_poster_tree = get_anime_poster(anime_genre,anime_id,anime_name)

        #(6)
        anime_story = get_anime_story(anime_poster_tree)

        #(7)
        studio, anime_source, anime_theme =  get_anime_studio_theme(anime_poster_tree)
        anime_origin = [studio, anime_source, anime_theme]
        all_themes = "_".join(anime_theme)

        #(8)
        anime_score = get_anime_score(anime_poster_tree)


        anime_all = [anime_id, anime_name, anime_score, status,
                     eps, run_time, brd, year,
                     all_genres, studio, anime_source, all_themes,
                     anime_poster_link, anime_link, anime_story]
        raw_data.append(anime_all)
        a+=1
    p+=1


 page(0), Anime(0): Shingeki_no_Kyojin
 page(0), Anime(1): Fullmetal_Alchemist__Brotherhood
 page(0), Anime(2): One_Punch_Man
 page(0), Anime(3): Sword_Art_Online
 page(0), Anime(4): Boku_no_Hero_Academia
 page(0), Anime(5): Kimetsu_no_Yaiba
 page(0), Anime(6): Naruto
 page(0), Anime(7): Tokyo_Ghoul
 page(0), Anime(8): Hunter_x_Hunter_2011
 page(0), Anime(9): Shingeki_no_Kyojin_Season_2
 page(0), Anime(10): Boku_no_Hero_Academia_2nd_Season
 page(0), Anime(11): Naruto__Shippuuden
 page(0), Anime(12): Shingeki_no_Kyojin_Season_3
 page(0), Anime(13): Jujutsu_Kaisen
 page(0), Anime(14): Code_Geass__Hangyaku_no_Lelouch
 page(0), Anime(15): One_Piece
 page(0), Anime(16): Noragami
 page(0), Anime(17): Boku_no_Hero_Academia_3rd_Season
 page(0), Anime(18): Shingeki_no_Kyojin_Season_3_Part_2
 page(0), Anime(19): Akame_ga_Kill
 page(0), Anime(20): Nanatsu_no_Taizai
 page(0), Anime(21): Mirai_Nikki_TV
 page(0), Anime(22): Mob_Psycho_100
 page(0), Anime(23): Sword_Art_Online_II
 page(0), Anime(24):

 page(1), Anime(185): God_Eater
 page(1), Anime(186): Evangelion__10_You_Are_Not_Alone
 page(1), Anime(187): Shakugan_no_Shana
 page(1), Anime(188): Gate__Jieitai_Kanochi_nite_Kaku_Tatakaeri_Part_2
 page(1), Anime(189): Musaigen_no_Phantom_World
 page(1), Anime(190): Boku_no_Hero_Academia_the_Movie_2__Heroes_Rising
 page(1), Anime(191): Gintama
 page(1), Anime(192): Darwins_Game
 page(1), Anime(193): Evangelion__20_You_Can_Not_Advance
 page(1), Anime(194): Ichiban_Ushiro_no_Daimaou
 page(1), Anime(195): AldnoahZero
 page(1), Anime(196): Zero_no_Tsukaima__Futatsuki_no_Kishi
 page(1), Anime(197): Sousei_no_Onmyouji
 page(1), Anime(198): Black_Lagoon__The_Second_Barrage
 page(1), Anime(199): Dorohedoro
 page(2), Anime(200): Full_Metal_Panic
 page(2), Anime(201): Shingeki_no_Kyojin__Kuinaki_Sentaku
 page(2), Anime(202): Vivy__Fluorite_Eyes_Song
 page(2), Anime(203): Rurouni_Kenshin__Meiji_Kenkaku_Romantan
 page(2), Anime(204): Re_Creators
 page(2), Anime(205): High_School_DxD_Hero
 page(2)

 page(3), Anime(363): Dungeon_ni_Deai_wo_Motomeru_no_wa_Machigatteiru_Darou_ka_IV__Shin_Shou_-_Meikyuu-hen
 page(3), Anime(364): Zero_kara_Hajimeru_Mahou_no_Sho
 page(3), Anime(365): Fate_Grand_Order__Zettai_Majuu_Sensen_Babylonia
 page(3), Anime(366): One_Piece_Film__Z
 page(3), Anime(367): Persona_4_the_Animation
 page(3), Anime(368): Saihate_no_Paladin
 page(3), Anime(369): Kimi_to_Boku_no_Saigo_no_Senjou_Aruiwa_Sekai_ga_Hajimaru_Seisen
 page(3), Anime(370): Full_Metal_Panic_The_Second_Raid
 page(3), Anime(371): Naruto__Shippuuden_Movie_2_-_Kizuna
 page(3), Anime(372): Devils_Line
 page(3), Anime(373): Pokemon_Movie_01__Mewtwo_no_Gyakushuu
 page(3), Anime(374): Darker_than_Black__Kuro_no_Keiyakusha_Gaiden
 page(3), Anime(375): Psycho-Pass_3
 page(3), Anime(376): Youjo_Senki_Movie
 page(3), Anime(377): Kyoukai_no_Kanata_Movie_2__Ill_Be_Here_-_Mirai-hen
 page(3), Anime(378): Gaikotsu_Kishi-sama_Tadaima_Isekai_e_Odekakechuu
 page(3), Anime(379): K__Missing_Kings
 page(3), Anime(380): K

 page(5), Anime(527): None
 page(5), Anime(528): Kimetsu_no_Yaiba__Katanakaji_no_Sato-hen
 page(5), Anime(529): Kaizoku_Oujo
 page(5), Anime(530): None
 page(5), Anime(531): C%C2%B3
 page(5), Anime(532): None
 page(5), Anime(533): Shuumatsu_no_Izetta
 page(5), Anime(534): Pokemon_Movie_03__Kesshoutou_no_Teiou_Entei
 page(5), Anime(535): Jujutsu_Kaisen_2nd_Season
 page(5), Anime(536): Boku_no_Hero_Academia__Sukue_Kyuujo_Kunren
 page(5), Anime(537): Vampire_Hunter_D_2000
 page(5), Anime(538): Heion_Sedai_no_Idaten-tachi
 page(5), Anime(539): Dragon_Crisis
 page(5), Anime(540): Berserk_2nd_Season
 page(5), Anime(541): Last_Exile
 page(5), Anime(542): Busou_Renkin
 page(5), Anime(543): Baki__Dai_Raitaisai-hen
 page(5), Anime(544): Sennen_Joyuu
 page(5), Anime(545): Trinity_Seven_Movie_1__Eternity_Library_to_Alchemic_Girl
 page(5), Anime(546): Tokyo_ESP
 page(5), Anime(547): Beatless
 page(5), Anime(548): Initial_D_Fourth_Stage
 page(5), Anime(549): Bubble
 page(5), Anime(550): Dragon_Ball_

 page(7), Anime(709): Black_Clover__Mahou_Tei_no_Ken
 page(7), Anime(710): Sidonia_no_Kishi__Daikyuu_Wakusei_Seneki
 page(7), Anime(711): Tsubasa_Chronicle_2nd_Season
 page(7), Anime(712): Hanyou_no_Yashahime__Sengoku_Otogizoushi
 page(7), Anime(713): RahXephon
 page(7), Anime(714): Noblesse__Awakening
 page(7), Anime(715): Tensei_shitara_Slime_Datta_Ken_Movie__Guren_no_Kizuna-hen
 page(7), Anime(716): Dragon_Ball_Z_Movie_09__Ginga_Girigiri_Bucchigiri_no_Sugoi_Yatsu
 page(7), Anime(717): Kingdom_2nd_Season
 page(7), Anime(718): Ueki_no_Housoku
 page(7), Anime(719): Senyuu
 page(7), Anime(720): Strike_the_Blood_III
 page(7), Anime(721): Dragon_Ball_Z_Movie_04__Super_Saiyajin_da_Son_Gokuu
 page(7), Anime(722): Senki_Zesshou_Symphogear
 page(7), Anime(723): None
 page(7), Anime(724): Mai-HiME
 page(7), Anime(725): Shingeki_no_Kyojin_Movie_1__Guren_no_Yumiya
 page(7), Anime(726): Hayate_no_Gotoku
 page(7), Anime(727): None
 page(7), Anime(728): Pokemon_Movie_08__Mew_to_Hadou_no_Yuusha_Luca

...

 page(346), Anime(33773): None
 page(346), Anime(33774): None
 page(346), Anime(33775): Nyanpire_The_Animation
 page(346), Anime(33776): Mato_Seihei_no_Slave
 page(346), Anime(33777): None
 page(346), Anime(33778): None
 page(346), Anime(33779): None
 page(346), Anime(33780): None
 page(346), Anime(33781): 4-nin_wa_Sorezore_Uso_wo_Tsuku
 page(346), Anime(33782): None
 page(346), Anime(33783): None
 page(346), Anime(33784): None
 page(346), Anime(33785): Ajin_Part_2__Shoutotsu
 page(346), Anime(33786): Kono_Danshi_Uchuujin_to_Tatakaemasu
 page(346), Anime(33787): None
 page(346), Anime(33788): Servamp_Movie__Alice_in_the_Garden
 page(346), Anime(33789): Soushin_Shoujo_Matoi
 page(346), Anime(33790): Nanatsu_no_Taizai__Ensa_no_Edinburgh
 page(346), Anime(33791): None
 page(346), Anime(33792): None
 page(346), Anime(33793): None
 page(346), Anime(33794): Yami_Shibai_5
 page(346), Anime(33795): Aa_Megami-sama_Sorezore_no_Tsubasa_Specials
 page(346), Anime(33796): Ajin_Part_3__Shougeki
 pag

 page(348), Anime(34004): None
 page(348), Anime(34005): Itou_Junji__Maniac
 page(348), Anime(34006): Wan_Sheng_Jie_2
 page(348), Anime(34007): None
 page(348), Anime(34008): None
 page(348), Anime(34009): Hi_no_Tori
 page(348), Anime(34010): None
 page(348), Anime(34011): Tezuka_Osamu_no_Buddha__Akai_Sabaku_yo_Utsukushiku
 page(348), Anime(34012): None
 page(348), Anime(34013): None
 page(348), Anime(34014): None
 page(348), Anime(34015): None
 page(348), Anime(34016): Dead_Mount_Death_Play
 page(348), Anime(34017): Mayonaka_no_Occult_Koumuin_OVA
 page(348), Anime(34018): None
 page(348), Anime(34019): None
 page(348), Anime(34020): None
 page(348), Anime(34021): None
 page(348), Anime(34022): None
 page(348), Anime(34023): None
 page(348), Anime(34024): None
 page(348), Anime(34025): None
 page(348), Anime(34026): None
 page(348), Anime(34027): Fate_strange_Fake_PV
 page(348), Anime(34028): None
 page(348), Anime(34029): None
 page(348), Anime(34030): None
 page(348), Anime(34031): N

 page(351), Anime(34315): None
 page(351), Anime(34316): Kyoufu_Shinbun_2014
 page(351), Anime(34317): None
 page(351), Anime(34318): None
 page(351), Anime(34319): None
 page(351), Anime(34320): None
 page(351), Anime(34321): None
 page(351), Anime(34322): None
 page(351), Anime(34323): None
 page(351), Anime(34324): None
 page(351), Anime(34325): None
 page(351), Anime(34326): Youkai_Watch_Movie_4__Shadow_Side_-_Oni-ou_no_Fukkatsu
 page(351), Anime(34327): None
 page(351), Anime(34328): None
 page(351), Anime(34329): Walking_Meat
 page(351), Anime(34330): Shinpi_no_Hou
 page(351), Anime(34331): None
 page(351), Anime(34332): None
 page(351), Anime(34333): None
 page(351), Anime(34334): None
 page(352), Anime(34335): None
 page(352), Anime(34336): None
 page(352), Anime(34337): Bai_Yao_Pu_3rd_Season
 page(352), Anime(34338): None
 page(352), Anime(34339): None
 page(352), Anime(34340): None
 page(352), Anime(34341): None
 page(352), Anime(34342): None
 page(352), Anime(34343): None
 p

 page(355), Anime(34644): None
 page(355), Anime(34645): None
 page(355), Anime(34646): None
 page(355), Anime(34647): None
 page(355), Anime(34648): None
 page(355), Anime(34649): None
 page(355), Anime(34650): None
 page(355), Anime(34651): None
 page(355), Anime(34652): None
 page(355), Anime(34653): None
 page(355), Anime(34654): None
 page(355), Anime(34655): None
 page(355), Anime(34656): None
 page(355), Anime(34657): None
 page(355), Anime(34658): None
 page(355), Anime(34659): None
 page(355), Anime(34660): None
 page(355), Anime(34661): None
 page(355), Anime(34662): None
 page(355), Anime(34663): None
 page(355), Anime(34664): None
 page(355), Anime(34665): None
 page(355), Anime(34666): None
 page(355), Anime(34667): None
 page(355), Anime(34668): None
 page(355), Anime(34669): None
 page(355), Anime(34670): None
 page(355), Anime(34671): None
 page(355), Anime(34672): None
 page(355), Anime(34673): None
 page(355), Anime(34674): None
 page(355), Anime(34675): None
 page(35

 page(358), Anime(34878): None
 page(358), Anime(34879): None
 page(358), Anime(34880): None
 page(358), Anime(34881): None
 page(358), Anime(34882): None
 page(358), Anime(34883): None
 page(358), Anime(34884): None
 page(358), Anime(34885): None
 page(358), Anime(34886): None
 page(358), Anime(34887): None
 page(358), Anime(34888): None
 page(358), Anime(34889): None
 page(358), Anime(34890): None
 page(358), Anime(34891): None
 page(358), Anime(34892): None
 page(358), Anime(34893): None
 page(358), Anime(34894): None
 page(358), Anime(34895): None
 page(358), Anime(34896): None
 page(358), Anime(34897): None
 page(358), Anime(34898): None


# Saving the data to CSV

In [21]:
# CSV headers
header_names = ['Anime ID', 'Anime Name','Score', 'Status',
               'Episodes', 'Run Time' ,'Broadcast Type', 'Year',
               'Genres', 'Studio','Source', 'Theme',
               'Poster Link' , 'Anime Link', 'Story']
df = pd.DataFrame.from_records(raw_data, columns = header_names)
df.to_csv(r'C:\Users\yassi\posters\animes.csv', sep=',')